## Business Understanding

### Data Understanding

## Data Exploration

### Rotten Tomatoes (Marlo)

### The Numbers (Andy)

### Box Office Mojo (Andy)

### TheMovieDB (John)

### IMDB (John)

## Visualizations